---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

<!-- After digesting the instructions, you can delete this cell, these are assignment instructions and do not need to be included in your final submission.  -->

{{< include instructions.qmd >}} 

# Code 

Provide the source code used for this section of the project here.

If you're using a package for code organization, you can import it at this point. However, make sure that the **actual workflow steps**—including data processing, analysis, and other key tasks—are conducted and clearly demonstrated on this page. The goal is to show the technical flow of your project, highlighting how the code is executed to achieve your results.

If relevant, link to additional documentation or external references that explain any complex components. This section should give readers a clear view of how the project is implemented from a technical perspective.

Remember, this page is a technical narrative, NOT just a notebook with a collection of code cells, include in-line Prose, to describe what is going on.

## Set the path to store data

In [1]:
import os  # Import os module for handling file paths

# Define the target folder for saving CSV files
output_folder_cleaned = "../../data/processed-data"
raw_data_folder = "../../data/raw-data"

# Create the folder if it doesn't exist
if not os.path.exists(output_folder_cleaned):
    os.makedirs(output_folder_cleaned)

## Processing and Cleaning Macroeconomic Data

In [2]:
import pandas as pd
import numpy as np
import os

# Load the raw data file
raw_data_file = os.path.join(raw_data_folder, "macro_series_raw_collection.csv")

try:
    # Read the CSV file
    merged_data = pd.read_csv(raw_data_file)
    merged_data["date"] = pd.to_datetime(merged_data["date"])  # Ensure the date column is in datetime format
except FileNotFoundError:
    print(f"Error: File '{raw_data_file}' not found.")
    exit()
except Exception as e:
    print(f"Error loading file: {e}")
    exit()

# Extract the year as a new column
merged_data["year"] = merged_data["date"].dt.year

# Check if the required columns exist
required_columns = ["GDP", "Umscent", "Exports", "Imports", "CPI"]
missing_columns = [col for col in required_columns if col not in merged_data.columns]
if missing_columns:
    print(f"Error: Missing required columns: {missing_columns}")
    exit()

# ======= Define Function =======

def fill_missing_with_annual_mean(data, column):
    """
    Fill missing values: Calculate the annual mean for each year and use it to fill missing values.
    """
    annual_mean = data.groupby("year")[column].transform("mean")
    data[column] = data[column].fillna(annual_mean)

# ======= Fill Missing Values =======

fill_missing_with_annual_mean(merged_data, "GDP")
fill_missing_with_annual_mean(merged_data, "Umscent")
fill_missing_with_annual_mean(merged_data, "Exports")
fill_missing_with_annual_mean(merged_data, "Imports")

# ======= Calculate Growth Rates =======

# Calculate Year-over-Year (YoY) and Month-over-Month (MoM) growth rates for GDP
merged_data["GDP_YoY"] = merged_data["GDP"].pct_change(periods=12)  # YoY GDP growth
merged_data["GDP_MoM"] = merged_data["GDP"].pct_change(periods=1)   # MoM GDP growth

# Calculate the YoY growth rate for CPI
merged_data["CPI_YoY"] = merged_data["CPI"].pct_change(periods=12)

# ======= Add Lag Variables =======

merged_data["GDP_Lag1"] = merged_data["GDP"].shift(1)  # Previous month's GDP
merged_data["CPI_Lag1"] = merged_data["CPI"].shift(1)  # Previous month's CPI

# ======= Filter Data =======
# Filter the data: Keep only data from the year 2000 and later
filtered_data = merged_data[merged_data["year"] >= 2000]

# ======= Save Cleaned Data =======
output_file = os.path.join(output_folder_cleaned, "macro_series_cleaned.csv")
filtered_data.to_csv(output_file, index=False)

# ======= Print Data Summary =======

print(f"Cleaned data saved to '{output_file}'.")
print(filtered_data.head(20))
print(filtered_data.info())

Cleaned data saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/macro_series_cleaned.csv'.
          date        GDP    CPI  Unemployment  FedFundsRate      M2  Umscent  \
640 2000-01-01  10002.179  169.3           4.0          5.45  4667.6    112.0   
641 2000-02-01  10250.952  170.0           4.1          5.73  4680.9    111.3   
642 2000-03-01  10250.952  171.0           4.0          5.85  4711.7    107.1   
643 2000-04-01  10247.720  170.9           3.8          6.02  4767.8    109.2   
644 2000-05-01  10250.952  171.2           4.0          6.27  4755.7    110.7   
645 2000-06-01  10250.952  172.2           4.0          6.53  4773.6    106.4   
646 2000-07-01  10318.165  172.7           4.0          6.54  4791.3    108.3   
647 2000-08-01  10250.952  172.7           4.1          6.50  4819.5    107.3   
648 2000-09-01  10250.952  173.6           3.9          6.52  4855.3    106.8   
649 2000-10-01  10435.744  173.9           3.9          6.51  487

## Processing Personal Income and Saving Data from BEA Table 20100

In [3]:
raw_data_file = os.path.join(raw_data_folder, "t20100_raw_data.csv")
t20100_data = pd.read_csv(raw_data_file)

# Filter key data
def clean_t20100_data(df):
    # Define the keywords for the fields we need
    keywords = [
        "Personal income",
        "Equals: Disposable personal income",
        "Equals: Personal saving",
        "Personal saving as a percentage of disposable personal income",
        "Personal consumption expenditures",
        "Less: Personal current taxes"
    ]
    # Filter the data we need
    filtered_df = df[df["LineDescription"].str.contains("|".join(keywords), case=False, na=False)]
    # Keep only the required fields
    filtered_df = filtered_df[["TimePeriod", "LineDescription", "DataValue"]]
    # Convert DataValue to numeric type
    
    filtered_df["DataValue"] = filtered_df["DataValue"].astype(str)  # Ensure it's a string
    filtered_df["DataValue"] = filtered_df["DataValue"].str.replace(",", "", regex=True).str.strip()  # Remove commas and spaces

    # Convert DataValue to numeric type
    filtered_df["DataValue"] = pd.to_numeric(filtered_df["DataValue"], errors="coerce")

    return filtered_df

filtered_t20100 = clean_t20100_data(t20100_data)

# Print the filtered data
print("Filtered T20100 data:")
print(filtered_t20100.head())


# Pivot the data to wide format
t20100_wide = filtered_t20100.pivot(index="TimePeriod", columns="LineDescription", values="DataValue").reset_index()

# Rename columns (for easier merging later)
t20100_wide.rename(columns={
    "Personal income": "Personal_Income (Millions of $)",
    "Equals: Disposable personal income": "Disposable_Income (Millions of $)",
    "Equals: Personal saving": "Personal_Saving (Millions of $)",
    "Personal saving as a percentage of disposable personal income": "Saving_Rate (%)",
    "Personal consumption expenditures": "Consumption_Expenditures (Millions of $)",
    "Less: Personal current taxes": "Personal_Taxes (Millions of $)"
}, inplace=True)

# Print the reshaped data
print("Reshaped T20100 data:")
print(t20100_wide.tail())

filtered_data = t20100_wide[t20100_wide["TimePeriod"] >= 2000]



file_name = os.path.join(output_folder_cleaned, f"personal_info_cleaned.csv")
filtered_data.to_csv(file_name, index=False)
print(f"Data for personal_info_cleaned saved to '{file_name}'.")

Filtered T20100 data:
   TimePeriod  LineDescription  DataValue
0        1929  Personal income    85289.0
1        1930  Personal income    76533.0
2        1931  Personal income    65668.0
3        1932  Personal income    50294.0
4        1933  Personal income    47234.0
Reshaped T20100 data:
LineDescription  TimePeriod  \
90                     2019   
91                     2020   
92                     2021   
93                     2022   
94                     2023   

LineDescription  Disposable personal income, chained (2017) dollars  \
90                                                             3.1    
91                                                             6.3    
92                                                             3.4    
93                                                            -5.5    
94                                                             5.1    

LineDescription  Disposable personal income, current dollars  \
90                        

## Processing Government Spending Data from BEA Table 10105

In [4]:
raw_data_file = os.path.join(raw_data_folder, "t10105_raw_data.csv")
t10105_data = pd.read_csv(raw_data_file)

def clean_t10105_data(df):
    # Define the keywords for the required fields
    keywords = [
       "Government consumption expenditures and gross investment",
       "Federal",
       "State and local"
    ]
    # Filter the required data
    filtered_df = df[df["LineDescription"].str.contains("|".join(keywords), case=False, na=False)]
    # Keep only the required columns
    filtered_df = filtered_df[["TimePeriod", "LineDescription", "DataValue"]]
    # Remove commas from the DataValue column (to prepare for numeric conversion)
    filtered_df["DataValue"] = filtered_df["DataValue"].str.replace(",", "", regex=True)
    # Convert the cleaned DataValue column to numeric type
    filtered_df["DataValue"] = pd.to_numeric(filtered_df["DataValue"], errors="coerce")
    return filtered_df

# Fetch T10105 data
filtered_t10105 = clean_t10105_data(t10105_data)

# Print the filtered data
print("Filtered T10105 data:")
print(filtered_t10105.head())

# Reshape the data to wide format
t10105_wide = filtered_t10105.pivot(index="TimePeriod", columns="LineDescription", values="DataValue").reset_index()

# Rename columns (for easier merging or analysis)
t10105_wide.rename(columns={
    "Government consumption expenditures and gross investment": "Government consumption expenditures and gross investment",
    
}, inplace=True)

# Convert the data to billions of dollars by dividing by 1000
# Add "(Millions of $)" to column names for better clarity
t10105_wide.rename(columns=lambda x: f"{x} (Millions of $)" if x != "TimePeriod" else x, inplace=True)

# Print the reshaped data
print("Reshaped T10105 data:")
print(t10105_wide.tail())

# Save the data as a CSV file
file_name = os.path.join(output_folder_cleaned, f"Government_Spending_Breakdown.csv")
t10105_wide.to_csv(file_name, index=False)
print(f"Data for Government_Spending_Breakdown saved to '{file_name}'.")

Filtered T10105 data:
      TimePeriod                                    LineDescription  DataValue
1995        1929  Government consumption expenditures and gross ...       9622
1996        1930  Government consumption expenditures and gross ...      10273
1997        1931  Government consumption expenditures and gross ...      10169
1998        1932  Government consumption expenditures and gross ...       8946
1999        1933  Government consumption expenditures and gross ...       8875
Reshaped T10105 data:
LineDescription  TimePeriod  Federal (Millions of $)  \
90                     2019                  1419500   
91                     2020                  1523016   
92                     2021                  1603184   
93                     2022                  1641036   
94                     2023                  1762627   

LineDescription  Government consumption expenditures and gross investment (Millions of $)  \
90                                                  

## Processing Market Data and Calculating Technical Indicators

In [5]:

def calculate_rsi(data, window=14):
    """
    Calculate RSI (Relative Strength Index) indicator.
    """
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def clean_and_process_data(data, date_range):
    """
    Cleans and processes raw financial data, including calculating technical indicators,
    filling missing values, and generating target variables.
    """
    try:
        # Ensure the 'Date' column is in datetime format and set it as the index
        data["Date"] = pd.to_datetime(data["Date"])
        data = data.set_index("Date")

        # === Calculate technical indicators ===
        data["MA_50"] = data["Close"].rolling(window=50).mean()  # 50-day moving average
        data["MA_200"] = data["Close"].rolling(window=200).mean()  # 200-day moving average
        data["Daily_Return"] = data["Close"].pct_change()  # Daily return
        data["Volatility"] = data["Daily_Return"].rolling(window=30).std()  # 30-day volatility
        data["RSI"] = calculate_rsi(data, window=14)  # RSI indicator
        data["EMA_12"] = data["Close"].ewm(span=12, adjust=False).mean()  # 12-day EMA
        data["EMA_26"] = data["Close"].ewm(span=26, adjust=False).mean()  # 26-day EMA
        data["MACD"] = data["EMA_12"] - data["EMA_26"]  # MACD line
        data["Signal_Line"] = data["MACD"].ewm(span=9, adjust=False).mean()  # Signal line
        data["High_Low"] = data["High"] - data["Low"]  # High-Low range
        data["High_Close"] = abs(data["High"] - data["Close"].shift(1))  # High-Close range
        data["Low_Close"] = abs(data["Low"] - data["Close"].shift(1))  # Low-Close range
        data["True_Range"] = data[["High_Low", "High_Close", "Low_Close"]].max(axis=1)  # True Range
        data["ATR"] = data["True_Range"].rolling(window=14).mean()  # Average True Range (ATR)
        data["OBV"] = (np.sign(data["Close"].diff()) * data["Volume"]).fillna(0).cumsum()  # On-Balance Volume (OBV)

        # Add lagged technical indicators
        for lag in [1, 2, 3]:
            data[f"RSI_Lag{lag}"] = data["RSI"].shift(lag)
            data[f"MACD_Lag{lag}"] = data["MACD"].shift(lag)
            data[f"Close_Lag{lag}"] = data["Close"].shift(lag)
            data[f"Volume_Lag{lag}"] = data["Volume"].shift(lag)

        # === Align data with the specified date range ===
        data = data.reindex(date_range)

        # === Drop rows with insufficient data for technical indicators ===
        # Drop rows where MA_200 or other long-window indicators are NaN
        data.dropna(subset=["MA_200", "EMA_12", "EMA_26"], inplace=True)

        # === Fill missing values ===
        # Fill price data
        data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
        data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
        data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
        data["Low"] = data["Low"].fillna(method="ffill").fillna(method="bfill")

        # Fill volume data
        data["Volume"] = data["Volume"].fillna(0)

        # Fill technical indicators
        data["MA_50"] = data["MA_50"].fillna(method="ffill")
        data["MA_200"] = data["MA_200"].fillna(method="ffill")
        data["Daily_Return"] = data["Daily_Return"].fillna(0)  # Default daily return to 0
        data["Volatility"] = data["Volatility"].fillna(0)  # Default volatility to 0
        data["RSI"] = data["RSI"].fillna(50)  # Default RSI to neutral value of 50
        data["MACD"] = data["MACD"].fillna(0)
        data["Signal_Line"] = data["Signal_Line"].fillna(0)
        data["ATR"] = data["ATR"].fillna(0)
        data["High_Low"] = data["High_Low"].fillna(0)
        data["High_Close"] = data["High_Close"].fillna(0)
        data["Low_Close"] = data["Low_Close"].fillna(0)
        data["True_Range"] = data["True_Range"].fillna(0)

        # Fill lagged indicators
        for lag in [1, 2, 3]:
            data[f"RSI_Lag{lag}"] = data[f"RSI_Lag{lag}"].fillna(50)
            data[f"MACD_Lag{lag}"] = data[f"MACD_Lag{lag}"].fillna(0)
            data[f"Close_Lag{lag}"] = data[f"Close_Lag{lag}"].fillna(method="ffill").fillna(method="bfill")
            data[f"Volume_Lag{lag}"] = data[f"Volume_Lag{lag}"].fillna(0)

        # === Generate target variables ===
        data["Target"] = np.where(data["Close"].shift(-5) > data["Close"], 1, 0)  # Binary target
        data["Future_Return_5D"] = data["Close"].shift(-5) / data["Close"] - 1  # 5-day future return

        # Fill target variables
        data["Target"] = data["Target"].fillna(-1)  # Mark missing targets as -1
        data["Future_Return_5D"] = data["Future_Return_5D"].fillna(0)

        # Reset index
        data.reset_index(inplace=True)

        # Check for any remaining missing values
        if data.isnull().values.any():
            print("Warning: There are still missing values after processing:")
            print(data.isnull().sum())

        return data
    except Exception as e:
        print(f"Error cleaning data: {e}")
        return None

if __name__ == "__main__":
    # Define date range
    start_date = "2000-11-01"
    end_date = "2024-11-30"
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Process raw files in the folder
    for file_name in os.listdir(raw_data_folder):
        if file_name.endswith("_raw_a.csv"):
            asset_name = file_name.replace("_raw_a.csv", "")
            raw_file_path = os.path.join(raw_data_folder, file_name)
            raw_data = pd.read_csv(raw_file_path)

            # Clean and process the data
            cleaned_data = clean_and_process_data(raw_data, date_range)
            if cleaned_data is not None:
                # Save cleaned data
                processed_file_path = os.path.join(output_folder_cleaned, f"{asset_name}_cleaned.csv")
                cleaned_data.to_csv(processed_file_path, index=False)
                print(f"Cleaned data for {asset_name} saved to '{processed_file_path}'.")

/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:62: FutureWarning: Series.fillna with 'meth

Cleaned data for Gold saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/Gold_cleaned.csv'.


/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:62: FutureWarning: Series.fillna with 'meth

Cleaned data for Nasdaq saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/Nasdaq_cleaned.csv'.


/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:62: FutureWarning: Series.fillna with 'meth

Cleaned data for Crude_Oil saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/Crude_Oil_cleaned.csv'.


/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:62: FutureWarning: Series.fillna with 'meth

Cleaned data for S&P_500 saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/S&P_500_cleaned.csv'.


/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Close"] = data["Close"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Open"] = data["Open"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["High"] = data["High"].fillna(method="ffill").fillna(method="bfill")
/var/folders/1z/n1k_hhl168s2rqld4kqxxr880000gn/T/ipykernel_78435/3783961959.py:62: FutureWarning: Series.fillna with 'meth

Cleaned data for Dow_Jones saved to '/Users/qqmian/Desktop/GU_5000/Stock_Market_Performance/data/processed-data/Dow_Jones_cleaned.csv'.
